# Evaluation of feature selection results

## Importing some packages

In [24]:
import os.path
import numpy as np
import pandas as pd

from scipy.stats import ttest_ind, ttest_rel
import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
from statsmodels.stats.contingency_tables import mcnemar

from config import *

## Loading the results

In [25]:
results= pd.read_csv('feature_selection_results.csv')
runtimes= pd.read_csv('feature_selection_runtimes.csv')

## The analysis

In [27]:
results

,Unnamed: 0,mi_3,mi_7,mi_11,mi_21,eqw_2,eqf_2,kmeans_2,distortion_aligned_2,eqw_5,...,kmeans_square-root,distortion_aligned_square-root,eqw_sturges-formula,eqf_sturges-formula,kmeans_sturges-formula,distortion_aligned_sturges-formula,eqw_rice-rule,eqf_rice-rule,kmeans_rice-rule,distortion_aligned_rice-rule
0,airfoil,1.302500,1.205000,1.130000,1.115000,1.147500,2.537500,1.140000,1.120000,1.092500,...,1.085000,1.082500,1.102500,2.180000,1.097500,1.100000,1.102500,2.097500,1.097500,1.087500
1,cpu_performance,2.321667,2.190000,2.285000,2.353333,2.405000,4.326667,2.470000,2.615000,2.435000,...,2.456667,2.461667,2.496667,4.111667,2.411667,2.355000,2.503333,3.751667,2.453333,2.463333
2,forestfires,8.932941,9.751765,9.988824,9.657059,10.465294,8.617647,10.461765,10.510000,10.891765,...,10.835882,9.747059,9.162941,7.284706,10.308824,10.652941,9.041176,7.810588,10.192353,10.039412
3,real_estate_valuation,2.960000,2.864000,2.808000,2.758000,2.980000,4.040000,2.106000,2.212000,2.160000,...,2.528000,2.468000,2.228000,3.358000,2.450000,2.394000,2.380000,3.198000,2.418000,2.496000
4,residential_building,8.136321,8.108868,8.598868,8.953208,9.989340,9.355566,9.790377,9.886509,10.175660,...,10.261698,10.174434,9.321132,10.134151,9.981792,10.081415,9.330849,10.144151,10.153679,10.236509
5,slump_test,3.765000,3.147500,3.525000,3.756250,3.045000,5.758750,3.008750,3.447500,3.368750,...,3.503750,3.502500,3.322500,4.657500,3.583750,3.500000,3.933750,4.423750,3.503750,3.502500
6,stock_portfolio_performance,2.790000,2.812000,2.754000,2.592000,2.592000,2.278000,2.494000,2.500000,2.584000,...,2.602000,2.678000,2.732000,2.116000,2.662000,2.682000,2.710000,2.070000,2.616000,2.678000
7,winequality_red,1.903000,1.697000,1.779000,1.746000,1.627000,3.722000,1.627000,1.869000,1.772000,...,1.678000,1.828000,1.824000,3.642000,1.824000,1.824000,1.824000,3.642000,1.824000,1.824000
8,winequality_white,2.897000,3.004000,3.332000,3.448000,1.676000,4.740000,1.676000,3.165000,1.688000,...,1.666000,3.072000,1.734000,3.961000,1.709000,1.709000,1.734000,3.961000,1.709000,1.709000
9,yacht_hydrodynamics,2.438000,2.438000,2.438000,2.444000,2.600000,5.404000,2.406000,2.504000,2.588000,...,2.068000,2.134000,2.134000,5.476000,2.184000,2.198000,2.138000,5.426000,2.112000,2.166000


In [28]:
results.columns

Index(['Unnamed: 0', 'mi_3', 'mi_7', 'mi_11', 'mi_21', 'eqw_2', 'eqf_2',
       'kmeans_2', 'distortion_aligned_2', 'eqw_5', 'eqf_5', 'kmeans_5',
       'distortion_aligned_5', 'eqw_square-root', 'eqf_square-root',
       'kmeans_square-root', 'distortion_aligned_square-root',
       'eqw_sturges-formula', 'eqf_sturges-formula', 'kmeans_sturges-formula',
       'distortion_aligned_sturges-formula', 'eqw_rice-rule', 'eqf_rice-rule',
       'kmeans_rice-rule', 'distortion_aligned_rice-rule'],
      dtype='object')

In [29]:
multiindex= pd.MultiIndex.from_tuples([('', 'database'), ('MI', 3), ('MI', 7), ('MI', 11), ('MI', 21), ('EQW', 2), ('EQF', 2), ('kmeans', 2), ('DA', 2), ('EQW', 5), ('EQF', 5), ('kmeans', 5), ('DA', 5), ('EQW', 'square-root'), ('EQF', 'square-root'), ('kmeans', 'square-root'), ('DA', 'square-root'), ('EQW', 'Struges-form.'), ('EQF', 'Struges-form.'), ('kmeans', 'Struges-form.'), ('DA', 'Struges-form.'), ('EQW', 'Rice-rule'), ('EQF', 'Rice-rule'), ('kmeans', 'Rice-rule'), ('DA', 'Rice-rule')])

In [30]:
results.columns=multiindex

In [38]:
results= results[[('', 'database'), ('MI', 3), ('MI', 7), ('MI', 11), ('MI', 21), ('EQW', 2), ('EQW', 5), ('EQW', 'square-root'), ('EQW', 'Struges-form.'),('EQW', 'Rice-rule'), ('EQF', 2), ('EQF', 5), ('EQF', 'square-root'), ('EQF', 'Struges-form.'), ('EQF', 'Rice-rule'), ('kmeans', 2), ('kmeans', 5), ('kmeans', 'square-root'), ('kmeans', 'Struges-form.'), ('kmeans', 'Rice-rule'), ('DA', 2), ('DA', 5), ('DA', 'square-root'), ('DA', 'Struges-form.'), ('DA', 'Rice-rule')]]

In [39]:
results

MI                                 \
                       database         3         7        11         21   
0                       airfoil  1.302500  1.205000  1.130000   1.115000   
1               cpu_performance  2.321667  2.190000  2.285000   2.353333   
2                   forestfires  8.932941  9.751765  9.988824   9.657059   
3         real_estate_valuation  2.960000  2.864000  2.808000   2.758000   
4          residential_building  8.136321  8.108868  8.598868   8.953208   
5                    slump_test  3.765000  3.147500  3.525000   3.756250   
6   stock_portfolio_performance  2.790000  2.812000  2.754000   2.592000   
7               winequality_red  1.903000  1.697000  1.779000   1.746000   
8             winequality_white  2.897000  3.004000  3.332000   3.448000   
9           yacht_hydrodynamics  2.438000  2.438000  2.438000   2.444000   
10                         ccpp  1.526667  1.526667  1.526667   1.526667   
11                  communities  6.513464  6.025490  5.673791   5.572745   
12                     diabetes  1.650000  1.680000  1.640000   1.540000   
13                        laser  1.756667  1.756667  1.756667   1.756667   
14                     autoMPG6  1.995000  1.950000  2.037500   1.900000   
15                       wizmir  2.576250  2.228750  2.152500   2.146250   
16                      wankara  2.067500  1.843750  1.863750   1.881250   
17                     mortgage  3.854286  3.767143  3.557143   4.042857   
18                     baseball  4.166000  4.147333  3.810667   3.412667   
19                        ele_1  1.000000  1.000000  1.000000   1.000000   
20                     treasury  3.796429  2.870714  2.783571   2.707143   
21                    compactiv  4.357000  4.679500  4.810500   5.049000   
22                      puma32h  8.631935  9.070000  9.722258  10.607097   

          EQW                                                 ...     kmeans  \
            2          5 square-root Struges-form. Rice-rule  ...          2   
0    1.147500   1.092500    1.100000      1.102500  1.102500  ...   1.140000   
1    2.405000   2.435000    2.511667      2.496667  2.503333  ...   2.470000   
2   10.465294  10.891765    9.160588      9.162941  9.041176  ...  10.461765   
3    2.980000   2.160000    2.550000      2.228000  2.380000  ...   2.106000   
4    9.989340  10.175660    9.450755      9.321132  9.330849  ...   9.790377   
5    3.045000   3.368750    3.933750      3.322500  3.933750  ...   3.008750   
6    2.592000   2.584000    2.570000      2.732000  2.710000  ...   2.494000   
7    1.627000   1.772000    1.999000      1.824000  1.824000  ...   1.627000   
8    1.676000   1.688000    1.838000      1.734000  1.734000  ...   1.676000   
9    2.600000   2.588000    2.200000      2.134000  2.138000  ...   2.406000   
10   1.526667   1.526667    1.526667      1.526667  1.526667  ...   1.526667   
11   9.108758   9.034902    8.487712      8.427908  8.487712  ...   9.195033   
12   1.620000   1.500000    1.530000      1.770000  1.790000  ...   1.710000   
13   1.756667   1.756667    1.756667      1.756667  1.756667  ...   1.756667   
14   1.855000   1.942500    1.995000      1.897500  1.917500  ...   1.865000   
15   2.291250   1.992500    2.073750      2.020000  2.063750  ...   2.140000   
16   2.068750   1.996250    1.908750      1.906250  1.932500  ...   2.026250   
17   3.337857   2.720714    2.945000      3.225714  3.282143  ...   3.400714   
18   5.154667   4.862000    3.108667      3.193333  3.278000  ...   4.962000   
19   1.000000   1.000000    1.000000      1.000000  1.000000  ...   1.000000   
20   3.311429   3.511429    3.510714      3.395000  3.515714  ...   3.512143   
21   4.525500   3.330500    4.086000      3.957000  4.086000  ...   4.202000   
22  13.649677   9.468065    8.145161      9.715161  9.313871  ...  13.291935   

                                                           DA             \
            5 square-root Struges-form.  Rice-rule          2

In [42]:
for c in results.columns[1:]:
    results[c]= results[c].apply(lambda x: np.round(x, 1))

In [43]:
results

MI                    EQW                    \
                       database    3    7    11    21     2     5 square-root   
0                       airfoil  1.3  1.2   1.1   1.1   1.1   1.1         1.1   
1               cpu_performance  2.3  2.2   2.3   2.4   2.4   2.4         2.5   
2                   forestfires  8.9  9.8  10.0   9.7  10.5  10.9         9.2   
3         real_estate_valuation  3.0  2.9   2.8   2.8   3.0   2.2         2.6   
4          residential_building  8.1  8.1   8.6   9.0  10.0  10.2         9.5   
5                    slump_test  3.8  3.1   3.5   3.8   3.0   3.4         3.9   
6   stock_portfolio_performance  2.8  2.8   2.8   2.6   2.6   2.6         2.6   
7               winequality_red  1.9  1.7   1.8   1.7   1.6   1.8         2.0   
8             winequality_white  2.9  3.0   3.3   3.4   1.7   1.7         1.8   
9           yacht_hydrodynamics  2.4  2.4   2.4   2.4   2.6   2.6         2.2   
10                         ccpp  1.5  1.5   1.5   1.5   1.5   1.5         1.5   
11                  communities  6.5  6.0   5.7   5.6   9.1   9.0         8.5   
12                     diabetes  1.6  1.7   1.6   1.5   1.6   1.5         1.5   
13                        laser  1.8  1.8   1.8   1.8   1.8   1.8         1.8   
14                     autoMPG6  2.0  2.0   2.0   1.9   1.9   1.9         2.0   
15                       wizmir  2.6  2.2   2.2   2.1   2.3   2.0         2.1   
16                      wankara  2.1  1.8   1.9   1.9   2.1   2.0         1.9   
17                     mortgage  3.9  3.8   3.6   4.0   3.3   2.7         2.9   
18                     baseball  4.2  4.1   3.8   3.4   5.2   4.9         3.1   
19                        ele_1  1.0  1.0   1.0   1.0   1.0   1.0         1.0   
20                     treasury  3.8  2.9   2.8   2.7   3.3   3.5         3.5   
21                    compactiv  4.4  4.7   4.8   5.0   4.5   3.3         4.1   
22                      puma32h  8.6  9.1   9.7  10.6  13.6   9.5         8.1   

                            ... kmeans                                  \
   Struges-form. Rice-rule  ...      2     5 square-root Struges-form.   
0            1.1       1.1  ...    1.1   1.1         1.1           1.1   
1            2.5       2.5  ...    2.5   2.4         2.5           2.4   
2            9.2       9.0  ...   10.5  10.2        10.8          10.3   
3            2.2       2.4  ...    2.1   2.3         2.5           2.4   
4            9.3       9.3  ...    9.8   9.3        10.3          10.0   
5            3.3       3.9  ...    3.0   3.4         3.5           3.6   
6            2.7       2.7  ...    2.5   2.6         2.6           2.7   
7            1.8       1.8  ...    1.6   1.8         1.7           1.8   
8            1.7       1.7  ...    1.7   1.7         1.7           1.7   
9            2.1       2.1  ...    2.4   2.4         2.1           2.2   
10           1.5       1.5  ...    1.5   1.5         1.5           1.5   
11           8.4       8.5  ...    9.2   8.1         7.9           8.0   
12           1.8       1.8  ...    1.7   1.8         1.8           1.8   
13           1.8       1.8  ...    1.8   1.8         1.8           1.8   
14           1.9       1.9  ...    1.9   1.9         1.9           2.0   
15           2.0       2.1  ...    2.1   2.1         2.1           2.1   
16           1.9       1.9  ...    2.0   1.9         1.9           1.9   
17           3.2       3.3  ...    3.4   3.3         2.9           3.1   
18           3.2       3.3  ...    5.0   3.2         3.3           3.3   
19           1.0       1.0  ...    1.0   1.0         1.0           1.0   
20           3.4       3.5  ...    3.5   3.5         3.4           3.5   
21           4.0       4.1  ...    4.2   3.4         3.0           3.1   
22           9.7       9.3  ...   13.3  12.6         8.6           8.9   

                DA                                            
   Rice-rule     2     5 square-root Struges-form. Rice-rule  
0        1.1   1.1   1.1       